# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [seldon-core Python package](https://pypi.org/project/seldon-core/) (```pip install seldon-core```)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

# Install Helm

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

## Start seldon-core

In [ ]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd --set usage_metrics.enabled=true

In [ ]:
!helm install ../helm-charts/seldon-core --name seldon-core --namespace seldon  --set ambassador.enabled=true

In [ ]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-seldon-apiserver
!kubectl rollout status deploy/seldon-core-ambassador

## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Serve Single Model

In [ ]:
!helm install ../helm-charts/seldon-single-model --name mymodel

In [ ]:
!helm template ../helm-charts/seldon-single-model --name mymodel | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-7cd068f

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_ambassador("mymodel","seldon",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("mymodel","seldon",API_AMBASSADOR)

In [ ]:
!helm delete mymodel --purge

## Serve AB Test

In [ ]:
!helm install ../helm-charts/seldon-abtest --name myabtest

In [ ]:
!helm template ../helm-charts/seldon-abtest --name myabtest | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/myabtest-abtest-41de5b8
!kubectl rollout status deploy/myabtest-abtest-df66c5c

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_ambassador("myabtest","seldon",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("myabtest","seldon",API_AMBASSADOR)

In [ ]:
!helm delete myabtest --purge

## Serve Multi-Armed Bandit

In [ ]:
!helm install ../helm-charts/seldon-mab --name mymab

In [ ]:
!helm template ../helm-charts/seldon-mab --name mymab | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymab-abtest-41de5b8
!kubectl rollout status deploy/mymab-abtest-b8038b2
!kubectl rollout status deploy/mymab-abtest-df66c5c 

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_ambassador("mymab","seldon",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("mymab","seldon",API_AMBASSADOR)

In [ ]:
!helm delete mymab --purge

## Serve Single Model with Outlier Detector

In [ ]:
!helm install ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true 

In [ ]:
!helm template ../helm-charts/seldon-single-model --name mymodel --set outlier_detection.enabled=true | pygmentize -l json

In [ ]:
!kubectl rollout status deploy/mymodel-mymodel-7cd068f
!kubectl rollout status deploy/mymodel-mymodel-d03fe8b

### Get predictions

In [ ]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [ ]:
r = rest_request_ambassador("mymodel","seldon",API_AMBASSADOR)
print(r.text)

#### gRPC Request

In [ ]:
grpc_request_ambassador("mymodel","seldon",API_AMBASSADOR)

In [ ]:
!helm delete mymodel --purge